In [10]:
import pandas as pd
import numpy as np
import json
from jinja2 import Environment, FileSystemLoader

In [11]:
def table_data_format(df_orig):
    metadata = []
    df = df_orig
    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for idx, col in enumerate(df.columns):
        helper = {}
        key = col
        if len(key) > 15:
            key = col[:15]

        helper['ascend'] = 0
        helper['name'] = key
        if(df.dtypes[col] == "object"):
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            helper['datatype'] = 'string'
            helper['values'] = df[col].unique().tolist()
        elif(df.dtypes[col] == "int64"):
            helper['datatype'] = 'int'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        elif(df.dtypes[col] == "float64"):
            helper['datatype'] = 'float'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        metadata.append(helper)

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value="missing value", inplace=True)
    data_array = []
    for row in range(df.shape[0]):
        aux = []
        for col in range(df.shape[1]):
            aux.append(df.iloc[row, col])
        data_array.append(aux)
    return metadata, data_array

In [18]:
path = r'C:\Users\Luis_Dutra\3D Objects\datasets\enem\enem_2019_for_table_viz.csv'

# Ler quantidade determinada de linhas do dataset
df = pd.read_csv(path,  encoding='iso8859-1')

df

,NU_INSCRICAO,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_REDACAO,Q001,Q002,Q006,Q025
0,190001043648,19,M,Solteiro(a),Parda,Brasileiro(a),RS,JÃ¡ concluÃ­ o Ensino MÃ©dio,2018,NÃ£o Respondeu,...,582.1,544.5,680.7,Espanhol,Sem problemas,760.0,Completou a PÃ³s-graduaÃ§Ã£o.,"Completou a Faculdade, mas nÃ£o completou a PÃ...",9 a 10 SM,Sim
1,190001684765,35,F,Casado(a)/Mora com companheiro(a),Branca,Brasileiro(a),RS,JÃ¡ concluÃ­ o Ensino MÃ©dio,Antes de 2007,NÃ£o Respondeu,...,538.5,546.8,367.3,Espanhol,Sem problemas,640.0,Completou a 8Âª sÃ©rie/9Âº ano do Ensino Funda...,"Completou a 4Âª sÃ©rie/5Âº ano, mas nÃ£o compl...",1 SM,Sim
2,190001684774,18,F,NÃ£o informado,Branca,Brasileiro(a),RS,Estou cursando e concluirei o Ensino MÃ©dio em...,NÃ£o informado,PÃºblica,...,529.1,526.9,412.6,InglÃªs,Sem problemas,660.0,"Completou o Ensino MÃ©dio, mas nÃ£o completou ...","Completou o Ensino MÃ©dio, mas nÃ£o completou ...",2.5 a 3 SM,Sim
3,190001684779,24,M,Solteiro(a),Branca,Brasileiro(a),RS,JÃ¡ concluÃ­ o Ensino MÃ©dio,2014,NÃ£o Respondeu,...,464.7,535.7,659.7,InglÃªs,Sem problemas,800.0,NÃ£o completou a 4Âª sÃ©rie/5Âº ano do Ensino ...,"Completou o Ensino MÃ©dio, mas nÃ£o completou ...",1 SM,Sim
4,190001684793,20,F,Solteiro(a),Branca,Brasileiro(a),RS,JÃ¡ concluÃ­ o Ensino MÃ©dio,2017,NÃ£o Respondeu,...,537.0,557.6,393.6,InglÃªs,Sem problemas,620.0,"Completou o Ensino MÃ©dio, mas nÃ£o completou ...",Completou a 8Âª sÃ©rie/9Âº ano do Ensino Funda...,2 a 2.5 SM,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21197,190006117762,18,M,Solteiro(a),Branca,Brasileiro(a),RS,Estou cursando e concluirei o Ensino MÃ©dio em...,NÃ£o informado,PÃºblica,...,420.6,545.6,650.4,Espanhol,Sem problemas,400.0,"Completou o Ensino MÃ©dio, mas nÃ£o completou ...","Completou o Ensino MÃ©dio, mas nÃ£o completou ...",3 a 4 SM,Sim
21198,190006117777,18,M,Solteiro(a),Branca,Brasileiro(a),RS,Estou cursando e concluirei o Ensino MÃ©dio em...,NÃ£o informado,PÃºblica,...,548.2,538.5,557.6,Espanhol,Sem problemas,560.0,"Completou o Ensino MÃ©dio, mas nÃ£o completou ...","Completou a Faculdade, mas nÃ£o completou a PÃ...",3 a 4 SM,Sim
21199,190006119127,16,M,Solteiro(a),Branca,Brasileiro(a),RS,Estou cursando e concluirei o Ensino MÃ©dio ap...,NÃ£o informado,NÃ£o Respondeu,...,425.3,520.5,604.2,Espanhol,Sem problemas,560.0,NÃ£o completou a 4Âª sÃ©rie/5Âº ano do Ensino ...,NÃ£o completou a 4Âª sÃ©rie/5Âº ano do Ensino ...,1 a 1.5 SM,Sim
21200,190006119133,17,M,Solteiro(a),Parda,Brasileiro(a),BA,Estou cursando e concluirei o Ensino MÃ©dio em...,NÃ£o informado,PÃºblica,...,461.0,558.2,586.5,InglÃªs,Parte desconectada,0.0,"Completou a 4Âª sÃ©rie/5Âº ano, mas nÃ£o compl...","Completou a Faculdade, mas nÃ£o completou a PÃ...",2 a 2.5 SM,Sim


In [19]:
del df['NU_INSCRICAO']

In [20]:
# Adiciona o  metadata_table_viz e dataarray_table_viz no html

file_loader = FileSystemLoader(r'C:\pandas-profiling\pandas_profiling\templates')
env = Environment(loader=file_loader)

template = env.get_template("table.html")

keys, data_array = table_data_format(df)

output = template.render(metadata_table_viz=keys, dataarray_table_viz=data_array)

# C:\pandas-profiling\notebook
with open("test.html", "w") as result_file:
    result_file.write(output)